In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.utils import to_categorical
import numpy as np
import os

data = pd.read_csv("dataset.csv")

image_paths = data["image_path"]
labels = data["artist_label"]

In [9]:
print(len(os.listdir('dataset/')))
num_artists = len(os.listdir('dataset/'))

4


In [10]:
def load_and_preprocess_images(image_paths, target_size=(128, 128)):
    images = []
    for img_path in image_paths:
        img = image.load_img(img_path, target_size=target_size)
        img_array = image.img_to_array(img)
        images.append(img_array)
    
    return np.array(images)

# Load and preprocess images
X = load_and_preprocess_images(image_paths)
X = X / 255.0  # Normalize the images to [0, 1] range

# Convert labels to numpy array and one-hot encode them
y = np.array(labels)
y = to_categorical(y, num_classes=num_artists)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Define a Convolutional Neural Network (CNN) model for multi-class classification
model = tf.keras.Sequential([
    # First convolutional block
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    
    # Second convolutional block
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    
    # Third convolutional block
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    
    # Flatten the output
    tf.keras.layers.Flatten(),
    
    # Fully connected layer
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    
    # Output layer with num_artists units for multi-class classification
    tf.keras.layers.Dense(num_artists, activation='softmax')  # num_artists classes, softmax activation
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',  # Categorical cross-entropy for multi-class classification
              metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_acc}")

# Predict on the test set
y_pred = np.argmax(model.predict(X_test), axis=1)  # Convert softmax probabilities to class labels

# Get the actual class labels (as integers)
y_true = np.argmax(y_test, axis=1)



c:\Users\zq\Documents\GitHub\yukien\venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
67/67 ━━━━━━━━━━━━━━━━━━━━ 13s 169ms/step - accuracy: 0.5327 - loss: 1.1367 - val_accuracy: 0.7038 - val_loss: 0.7861
Epoch 2/10
67/67 ━━━━━━━━━━━━━━━━━━━━ 12s 175ms/step - accuracy: 0.7106 - loss: 0.7721 - val_accuracy: 0.7509 - val_loss: 0.6404
Epoch 3/10
67/67 ━━━━━━━━━━━━━━━━━━━━ 12s 182ms/step - accuracy: 0.7726 - loss: 0.5789 - val_accuracy: 0.8000 - val_loss: 0.5804
Epoch 4/10
67/67 ━━━━━━━━━━━━━━━━━━━━ 11s 168ms/step - accuracy: 0.8146 - loss: 0.4853 - val_accuracy: 0.8189 - val_loss: 0.5449
Epoch 5/10
67/67 ━━━━━━━━━━━━━━━━━━━━ 11s 161ms/step - accuracy: 0.8436 - loss: 0.4314 - val_accuracy: 0.8509 - val_loss: 0.5481
Epoch 6/10
67/67 ━━━━━━━━━━━━━━━━━━━━ 11s 159ms/step - accuracy: 0.8893 - loss: 0.3266 - val_accuracy: 0.8491 - val_loss: 0.5522
Epoch 7/10
67/67 ━━━━━━━━━━━━━━━━━━━━ 11s 159ms/step - accuracy: 0.9083 - loss: 0.2717 - val_accuracy: 0.8717 - val_loss: 0.5554
Epoch 8/10
67/67 ━━━━━━━━━━━━━━━━━━━━ 11s 159ms/step - accuracy: 0.9193 - loss: 0.2195 - val_accu

ValueError: Number of classes, 4, does not match size of target_names, 3. Try specifying the labels parameter

In [13]:
# Classification report for performance evaluation
report = classification_report(y_true, y_pred, 
                              target_names=[f'Class {i}' for i in np.unique(y_true)])

print(report)

              precision    recall  f1-score   support

     Class 0       0.84      0.88      0.86       180
     Class 1       0.91      0.70      0.79        43
     Class 2       0.84      0.91      0.87       264
     Class 3       0.76      0.37      0.50        43

    accuracy                           0.84       530
   macro avg       0.84      0.72      0.76       530
weighted avg       0.84      0.84      0.83       530



precision    recall  f1-score   support

     Class 0       0.84      0.88      0.86       180
     Class 1       0.91      0.70      0.79        43
     Class 2       0.84      0.91      0.87       264
     Class 3       0.76      0.37      0.50        43

    accuracy                           0.84       530
   macro avg       0.84      0.72      0.76       530
weighted avg       0.84      0.84      0.83       530